In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import joblib
import math
import numpy as np
import itertools 
from itertools import product
from collections import Counter
import garpar as gp
import re
import skcriteria as skc
from skcriteria import pipeline
from skcriteria.preprocessing import invert_objectives, weighters, scalers
from skcriteria.madm import similarity, moora, electre
from skcriteria.ranksrev import RankInvariantChecker
from garpar.datasets import make_multisector, RissoLevyStable, RissoNormal, RissoUniform
import warnings

/home/nadia/indicador/lib/python3.10/site-packages/skcriteria/pipeline.py:27: SKCriteriaDeprecationWarning: The 'skcriteria.pipeline' module is deprecated since 0.9 and will be removed in 1.0 Use 'skcriteria.pipelines' instead.
  deprecate.warn(
/home/nadia/indicador/lib/python3.10/site-packages/skcriteria/madm.py:30: SKCriteriaDeprecationWarning: 'skcriteria.madm' module is deprecated, use 'skcriteria.agg' instead
  utils.deprecate.warn(deprecation_reason)
/home/nadia/indicador/lib/python3.10/site-packages/skcriteria/agg/similarity.py:27: SKCriteriaDeprecationWarning: The module 'skcriteria.agg.similarity' is deprecated since v0.9 and will be removed in v1.0. Please use 'skcriteria.agg.topsis' instead.
  deprecate.warn(


In [2]:
#--- 1. Configuración de Parámetros Fijos (CORREGIDO) ---
NUM_ITERACIONES = 100
STOCKS_TOTALES = 6  # 💡 CAMBIO CLAVE: Un solo stock por sector
DIAS_SIMULACION = 7
VENTANAS_A_PROBAR = [3, 5, 7] 

# --- 2. Definición de Parámetros y PRECIOS PERSONALIZADOS (CORREGIDO) ---
PRECIOS_INICIALES_TOTAL = [
    200.0,  # alua
    300.0,  # Precio para el Stock 2 (Sector 2)
    150.0,
    100,
    100,
    800# Precio para el Stock 3 (Sector 3)
] 

# --- 3. Definición de Parámetros de Sectores (Sin Cambios) ---
params_A = {"alpha": 1.6411, "beta": -0.0126, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_B = {"alpha": 2, "beta": -0.5, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_C = {"alpha": 1.9, "beta": 0.1, "mu": 0.001, "sigma": 0.004, "entropy": 0.6}
params_D = {"alpha": 1.6411, "beta": -0.0126, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_E = {"alpha": 1.6411, "beta": -0.0126, "mu": 0.0005, "sigma": 0.005, "entropy": 0.5}
params_F = {"alpha": 1.9, "beta": 0.1, "mu": 0.001, "sigma": 0.004, "entropy": 0.6}

parametros_sectores_base = [params_A, params_B, params_C, params_D, params_E]

# Diccionario para guardar todos los DataFrames: {Ventana: [df_corrida_1, df_corrida_2, ...]}
resultados_por_ventana = {ventana: [] for ventana in VENTANAS_A_PROBAR}

# --- 4. BUCLE DE SIMULACIÓN Y ALMACENAMIENTO ---

for VENTANA in VENTANAS_A_PROBAR:
    print(f"Iniciando {NUM_ITERACIONES} simulaciones para Ventana: {VENTANA} días...")
    
    for i in range(NUM_ITERACIONES):
        
        # Cálculo del random_state secuencial por bloque (i * 10000 + j * 100)
        random_seed_base = i * 10000 
        distribuciones_para_corrida = []
        
        for j, params in enumerate(parametros_sectores_base):
            semilla_sector = random_seed_base + (j * 100)
            params_corrida = params.copy() 
            params_corrida['random_state'] = semilla_sector
            distribuciones_para_corrida.append(RissoLevyStable(**params_corrida))

        # SIMULACIÓN DEL MERCADO
        stocksset = make_multisector(
            *distribuciones_para_corrida,
            stocks=STOCKS_TOTALES,
            days=DIAS_SIMULACION,
            window_size=VENTANA,
            price=PRECIOS_INICIALES_TOTAL 
        )
        
        # ALMACENAMIENTO DEL DATAFRAME DE PRECIOS
        df_simulacion = stocksset._prices_df 
        resultados_por_ventana[VENTANA].append(df_simulacion)
    
print("\n--- ¡Generación de Simulaciones Finalizada! ---")

Iniciando 100 simulaciones para Ventana: 3 días...
Iniciando 100 simulaciones para Ventana: 5 días...
Iniciando 100 simulaciones para Ventana: 7 días...

--- ¡Generación de Simulaciones Finalizada! ---


In [3]:
resultados_por_ventana

{3: [Stocks  rissolevystable_1_S0  rissolevystable_1_S1  rissolevystable_2_S0  \
  Days                                                                       
  0                 200.000000            300.000000            150.000000   
  1                 200.013163            300.009547            150.004554   
  2                 200.017952            300.017101            150.009037   
  3                 200.021572            300.015353            150.012329   
  4                 200.024124            300.025281            150.021387   
  5                 200.024549            300.039775            150.014714   
  6                 200.024915            300.039403            150.005393   
  7                 200.022983            300.029377            150.000311   
  
  Stocks  rissolevystable_3_S0  rissolevystable_4_S0  rissolevystable_5_S0  
  Days                                                                      
  0                 100.000000            100.000000        

In [4]:
# --- 0. CONFIGURACIÓN INICIAL ---
# Se recomienda suprimir el warning común de garpar/pypfopt sobre matrices no positivas
warnings.filterwarnings('ignore', category=UserWarning, message='The covariance matrix is non positive semidefinite. Amending eigenvalues.')
warnings.filterwarnings('ignore', category=UserWarning, message="No risk_free_rate specified, coercing it")
# ⚠️ DEFINICIÓN DE LOS OPTIMIZADORES A PROBAR (Asumida)
optimizadores_a_probar = {
    "max_sharpe": gp.optimize.mean_variance.MVOptimizer(model='max_sharpe'),
    "markowitz_target_return": gp.optimize.mean_variance.Markowitz(target_return=0.001),
    "markowitz_target_risk1": gp.optimize.mean_variance.Markowitz(target_risk=0.30),
    "markowitz_target_risk2": gp.optimize.mean_variance.Markowitz(target_risk=0.25),
}

rows = []

print("--- INICIO DEL PROCESAMIENTO DE MÉTRICAS (Usando StocksSet) ---")

# --- 1. BUCLE DE PROCESAMIENTO SOBRE LOS RESULTADOS GUARDADOS ---

for VENTANA, lista_simulaciones in resultados_por_ventana.items():
    
    for i, df_corrida in enumerate(lista_simulaciones):
        
        corrida_id = i + 1
        stocks_number_base = len(df_corrida.columns)

        # 💡 MEJORA DE ROBUSTEZ: Intentar crear el StocksSet primero
        stocksset_corrida = None
        try:
            stocksset_corrida = gp.StocksSet.from_prices(df_corrida, window_size=VENTANA)
        except Exception as e:
            # Si el StocksSet falla (ej: datos malformados), registra el fallo para TODOS los modelos
            print(f"⚠️ Error al crear StocksSet (Corrida {corrida_id}, Ventana {VENTANA}): {e}. Saltando modelos.")
            
            for opt_name in optimizadores_a_probar.keys():
                rows.append({
                    "Ventana": VENTANA,
                    "Corrida": corrida_id,
                    "Modelo_Opt": opt_name,
                    "stocks_number": stocks_number_base, 
                    "Cross_Entropy": np.nan,
                    "Zheng_Entropy": np.nan,
                    "Div_Ratio": np.nan,
                    "CV_MC": np.nan,
                    "PDI": np.nan
                })
            continue # Pasa a la siguiente corrida
        
        # 2. ITERACIÓN SOBRE LOS MODELOS DE OPTIMIZACIÓN
        for opt_name, optimizador in optimizadores_a_probar.items():
            
            try:
                # 3. OPTIMIZACIÓN + PRUNING
                mopt = optimizador.optimize(stocksset_corrida).weights_prune()
                
                # 4. CÁLCULO DE MÉTRICAS (EXITOSO)
                metricas = {
                    "stocks_number": len(mopt.stocks),
                    "Cross_Entropy": mopt.div.cross_entropy(),
                    "Zheng_Entropy": mopt.div.zheng_entropy(),
                    "Div_Ratio": mopt.div.ratio(),
                    "CV_MC": (mopt.div.mrc() * mopt.weights).std() / (mopt.div.mrc() * mopt.weights).mean(),
                    "PDI": mopt.div.pdi(n_components=3, random_state=100)
                }

                # 5. GUARDAR RESULTADO EXITOSO
                rows.append({
                    "Ventana": VENTANA,
                    "Corrida": corrida_id,
                    "Modelo_Opt": opt_name, 
                    **metricas
                })

            except Exception as e:
                # 🛠️ SOLUCIÓN AL KEYERROR: Garantizar que la fila de identificadores se guarde
                rows.append({
                    "Ventana": VENTANA,
                    "Corrida": corrida_id,
                    "Modelo_Opt": opt_name,
                    "stocks_number": stocks_number_base, 
                    "Cross_Entropy": np.nan,
                    "Zheng_Entropy": np.nan,
                    "Div_Ratio": np.nan,
                    "CV_MC": np.nan,
                    "PDI": np.nan
                })
                # print(f"⚠️ Error en {opt_name} (Corrida {corrida_id}, Ventana {VENTANA}): {e}") 
            
# --- 2. CREACIÓN DEL DATAFRAME FINAL ---

df_metricas_simulacion = pd.DataFrame(rows)

# Esta línea ahora es segura, ya que 'rows' garantiza las claves de índice
df_metricas_simulacion = df_metricas_simulacion.set_index(["Ventana", "Corrida", "Modelo_Opt"])

print("\n--- ANÁLISIS COMPLETADO ---")
print(f"Total de resultados guardados: {len(df_metricas_simulacion)} filas.")
print(df_metricas_simulacion.head(9))


--- INICIO DEL PROCESAMIENTO DE MÉTRICAS (Usando StocksSet) ---


/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10/site-packages/cvxpy/problems/problem.py:1539: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(
/home/nadia/indicador/lib/python3.10


--- ANÁLISIS COMPLETADO ---
Total de resultados guardados: 1200 filas.
                                         stocks_number  Cross_Entropy  \
Ventana Corrida Modelo_Opt                                              
3       1       max_sharpe                           5       4.913730   
                markowitz_target_return              6       0.119852   
                markowitz_target_risk1               5      13.992721   
                markowitz_target_risk2               5      14.007002   
        2       max_sharpe                           6       3.003324   
                markowitz_target_return              6       1.930387   
                markowitz_target_risk1               6      10.821468   
                markowitz_target_risk2               6      14.313074   
        3       max_sharpe                           6       9.529105   

                                         Zheng_Entropy  Div_Ratio     CV_MC  \
Ventana Corrida Modelo_Opt                   

In [14]:
df_metricas_simulacion[(df_metricas_simulacion.index.get_level_values('Ventana') == 3) &
    (df_metricas_simulacion.index.get_level_values('Corrida') ==10)]

stocks_number  Cross_Entropy  \
Ventana Corrida Modelo_Opt                                              
3       10      max_sharpe                           6            NaN   
                markowitz_target_return              6       0.000299   
                markowitz_target_risk1               5      13.736235   
                markowitz_target_risk2               5      13.761512   

                                         Zheng_Entropy  Div_Ratio     CV_MC  \
Ventana Corrida Modelo_Opt                                                    
3       10      max_sharpe                         NaN        NaN       NaN   
                markowitz_target_return   1.791465e+00   0.104872  1.394632   
                markowitz_target_risk1    4.274902e-07   0.062994  2.236068   
                markowitz_target_risk2    4.149959e-07   0.062994  2.236068   

                                              PDI  
Ventana Corrida Modelo_Opt                         
3       10      max_sharpe                    NaN  
                markowitz_target_return  1.301219  
                markowitz_target_risk1   1.304506  
                markowitz_target_risk2   1.304506

In [15]:
df_metricas_simulacion[(df_metricas_simulacion.index.get_level_values('Ventana') == 3) &
    (df_metricas_simulacion.index.get_level_values('Corrida') ==10)].dropna()

stocks_number  Cross_Entropy  \
Ventana Corrida Modelo_Opt                                              
3       10      markowitz_target_return              6       0.000299   
                markowitz_target_risk1               5      13.736235   
                markowitz_target_risk2               5      13.761512   

                                         Zheng_Entropy  Div_Ratio     CV_MC  \
Ventana Corrida Modelo_Opt                                                    
3       10      markowitz_target_return   1.791465e+00   0.104872  1.394632   
                markowitz_target_risk1    4.274902e-07   0.062994  2.236068   
                markowitz_target_risk2    4.149959e-07   0.062994  2.236068   

                                              PDI  
Ventana Corrida Modelo_Opt                         
3       10      markowitz_target_return  1.301219  
                markowitz_target_risk1   1.304506  
                markowitz_target_risk2   1.304506

In [6]:
df_metricas_simulacion[df_metricas_simulacion.index.get_level_values('Ventana') == 3].describe()

,stocks_number,Cross_Entropy,Zheng_Entropy,Div_Ratio,CV_MC,PDI
count,400.000000,3.800000e+02,3.800000e+02,380.000000,380.000000,380.000000
mean,5.360000,7.890105e+00,5.699991e-01,0.117072,1.995151,1.708851
std,0.939471,5.738855e+00,6.922279e-01,0.111695,0.542932,0.300144
min,3.000000,7.642667e-07,7.417216e-08,0.062993,0.208312,1.062941
25%,5.000000,9.064513e-01,8.929571e-07,0.062994,1.642322,1.479654
50%,6.000000,1.077958e+01,2.243373e-05,0.062994,2.165111,1.718754
75%,6.000000,1.340888e+01,1.120779e+00,0.122113,2.449489,1.918775
max,6.000000,1.471688e+01,1.791759e+00,1.097060,4.760203,2.595579


In [7]:
objectives = [
    max, #     'stocks_seleccionados', coincide con definición mayor número de stocks
    min, #     'Entropia_cruzada', mide el desorden al compararlo con una estrategia teórica de risk part equally
    max, #     'Entropia_Zheng', 
    max, #     'Ratio_diversific', mide la diversificación ganada por  mantener activos no correlacionados
    min, #     'CV_Contribuciones_marginales', si todos los activos tienen igual contribución marginal es un portfolio diversificado
    max, #     'Indice_diversific_port, si hay correlacióan alta el índice dará un valor bajo mientras que si no hay correlación el índice tenderá al valor N
]
dm = skc.mkdm(df_metricas_simulacion[df_metricas_simulacion.index.get_level_values('Ventana') == 3].to_numpy(), 
              objectives, alternatives=df_metricas_simulacion[df_metricas_simulacion.index.get_level_values('Ventana') == 3].index, 
              criteria=df_metricas_simulacion[df_metricas_simulacion.index.get_level_values('Ventana') == 3].columns)

In [8]:
pipe_igual_vector = pipeline.mkpipe(
    invert_objectives.NegateMinimize(),
    scalers.VectorScaler(target="matrix"),
    weighters.EqualWeighter(),
    similarity.TOPSIS(),
)
pipe_igual_vector
result_igual_vector = pipe_igual_vector.evaluate(dm)
result_igual_vector

/home/nadia/indicador/lib/python3.10/site-packages/skcriteria/utils/rank.py:67: RuntimeWarning: invalid value encountered in cast
  return stats.rankdata(arr, "dense").astype(np.int64)


ValueError: The data [-9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -9223372036854775808 -9223372036854775808 -9223372036854775808
 -92233720368547758